# LELEC2910 - Project part I

This notebook serves as a tutorial for the **first part** of the project. It complements the detailed guidelines for the project that are available on Moodle.

In [ ]:
# import required
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.signal as signal

import utils #utils.py, gathers useful functions for loading and plotting Alphasat measurements

## 1) Load Alphasat measurements 
The cell below loads Alphasat measurements for a chosen month (either 'september2019', 'december2020', 'march2021', 'june2021'). Four months of data are provided, one per season. They do not come from the same year as the Alphasat receiver sometimes suffer from failure events, preventing the use of its data. Hence, the months have been chosen in order to give you data of good quality, i.e., without too many missing days.

In [ ]:
# Load Alphasat measurements
month_name = 'march2021' #'september2019'#'march2021'#'december2020'#'june2021'
df_ch1, df_ch2, df_ch3, df_ch4 = utils.load_data('data',month_name)

# Print the dataframe associated to channel 2, to show you its content
print(df_ch2)

## 2) Rain and failure events
The identification of rain events has already been conducted, using visual inspection. Start and end dates of rain events are gathered in the events.csv file, loaded in the cell below. Then, a flag is added to the measurement dataframes, using the following convention: 0 for data under clear sky, 1 for data during a rain event, 2 for a failure of the system.

In [ ]:
# Load event file
event_file = 'events.csv'
df_event = pd.read_csv(event_file)

# Print the loaded event file to show you its content
print(df_event)

In [ ]:
# Addition of a flag, based on the recorded events
df_ch1, df_ch2, df_ch3, df_ch4 = utils.add_flags(df_event, df_ch1, df_ch2, df_ch3, df_ch4)

# Print of the second channel dataframe after the flag addition
print(df_ch1)

### Plot of loaded data
The cell below plots the loaded data for each day in the selected date range. It uses the plot_one_day_4ch() function defined in utils.py. The last argument of the function is a boolean to set to 'true' to save the figure for each day (in the /figure folder). You are encouraged to run the plots for all dates in the chosen month, and then skim through the saved figures. On each figure, rain events are depicted in red, while failure events are in grey.

In [ ]:
# Choose the start and end dates according to the month
all_dates = pd.date_range(start='2021-03-01', end='2021-03-31')

date = all_dates[16]

utils.plot_one_day_4ch(date, df_ch1, df_ch2, df_ch3, df_ch4, True)


## 3) Excess attenuation and statistics
It is now your turn to process the data and extract the excess attenuation for each month, at each frequency. Start with the excess attenuation time series, and then compute its statistics (CCDF). Remember to first define a template to determine the 0dB level.

**Make sure to remove failure events when you compute the statistics (especially for June 2021).**

In [ ]:
### TO DO, extraction of excess attenuation statistics for each month

# Compute the excess attenuation statistics for each channel
stats = utils.compute_stats(df_ch1, df_ch2, df_ch3, df_ch4)

# Print the statistics for channel 2
print(stats[0])

plt.plot(df_ch1['signal'].values)

# Apply a low-pass filter to remove high frequencies
b, a = signal.butter(4, 0.0001, btype='low', analog=False)
filtered_signals = utils.low_pass_filters(b, a, df_ch1, df_ch2, df_ch3, df_ch4)
flagged_signal = utils.flagging(filtered_signals, df_ch1, df_ch2, df_ch3, df_ch4, stats) # this is the template, maybe changes are needed

plt.plot(filtered_signals[0])
plt.plot(np.full(len(df_ch1['signal'].values), stats[0]['mean_signal']))
plt.show()

# Compute the excess attenuation statistics
excess = df_ch1['signal'].values - flagged_signal[0]

plt.plot(excess, label='Excess attenuation')
plt.plot(np.full(len(df_ch1['signal'].values), stats[0]['std_signal']), color='red', linestyle='dashed', label='Standard deviation')
plt.plot(np.full(len(df_ch1['signal'].values), -stats[0]['std_signal']), color='red', linestyle='dashed')
plt.legend()
plt.show()



## 4) Comparison with RAPIDS outputs
The cell below loads RAPIDS simulation outputs at LLN for the 19.7GHz-frequency, and is given as an example. Only the total attenuation and the rain attenuation are loaded. Per frequency, two files are needed (one file is for the elevation angles $\theta = 5°,10°,...,50°$ and the other is for $\theta=55°,...,90°$). Plots are given as examples, and should compare with the graphs generated automatically by the RAPIDS software (available in the output files).

In [ ]:
### Total attenuation at 19.7 GHz for LLN
field_names = ["SITE","SATELLITE","FREQUENCY","ELEVATION","EP","PROBABILITY","ATTENUATION"] #for total attenuation

file1 = pd.read_csv("rapids/lln_19GHz_1/output/ascii/attenuation_total.csv",skiprows=7,names=field_names)
file2 = pd.read_csv("rapids/lln_19GHz_2/output/ascii/attenuation_total.csv",skiprows=7,names=field_names)
rapids_total_attenuation_19_7_LLN = pd.concat((file1,file2))

print(rapids_total_attenuation_19_7_LLN)
utils.plot_RAPIDS_outputs(rapids_total_attenuation_19_7_LLN,"Total attenuation for LLN at 19.7 GHz")


### Rain attenuation at 19.7 GHz for LLN
field_names = ["SITE","SATELLITE","FREQUENCY","ELEVATION","ELEVATION_PROBABILITY","PROBABILITY","HR","R001","K","ALPHA","ATTENUATION"] #for rain

file1 = pd.read_csv("rapids/lln_19GHz_1/output/ascii/attenuation_rain.csv",skiprows=7,names=field_names)
file2 = pd.read_csv("rapids/lln_19GHz_2/output/ascii/attenuation_rain.csv",skiprows=7,names=field_names)
rapids_rain_attenuation_19_7_LLN = pd.concat((file1,file2))

print(rapids_rain_attenuation_19_7_LLN)
utils.plot_RAPIDS_outputs(rapids_rain_attenuation_19_7_LLN,"Rain attenuation for LLN at 19.7 GHz")


At this point, you should be able to compare your measured statistics with the ones from RAPIDS, knowing the elevation angle of the Alphasat satellite as seen from Louvain-la-Neuve (see slides of the project introduction session).

In [ ]:
### Total attenuation at 37 GHz for ljubljana

field_names = ["SITE","SATELLITE","FREQUENCY","ELEVATION","EP","PROBABILITY","ATTENUATION"] #for total attenuation

file1 = pd.read_csv("rapids/ljubljana_37GHz_1/output/ascii/attenuation_total.csv",skiprows=7,names=field_names)
file2 = pd.read_csv("rapids/ljubljana_37GHz_2/output/ascii/attenuation_total.csv",skiprows=7,names=field_names)

rapids_total_attenuation_37_LUB = pd.concat((file1,file2))

utils.plot_RAPIDS_outputs(rapids_total_attenuation_37_LUB,"Total attenuation for Ljubljana at 37 GHz")

### Rain attenuation at 37 GHz for ljubljana

field_names = ["SITE","SATELLITE","FREQUENCY","ELEVATION","ELEVATION_PROBABILITY","PROBABILITY","HR","R001","K","ALPHA","ATTENUATION"] #for rain

file1 = pd.read_csv("rapids/ljubljana_37GHz_1/output/ascii/attenuation_rain.csv",skiprows=7,names=field_names)
file2 = pd.read_csv("rapids/ljubljana_37GHz_2/output/ascii/attenuation_rain.csv",skiprows=7,names=field_names)

rapids_rain_attenuation_37_LUB = pd.concat((file1,file2))

utils.plot_RAPIDS_outputs(rapids_rain_attenuation_37_LUB,"Rain attenuation for Ljubljana at 37 GHz")

In [ ]:
### Total attenuation at 37 GHz for lln

field_names = ["SITE","SATELLITE","FREQUENCY","ELEVATION","EP","PROBABILITY","ATTENUATION"] #for total attenuation

file1 = pd.read_csv("rapids/lln_37GHz_1/output/ascii/attenuation_total.csv",skiprows=7,names=field_names)
file2 = pd.read_csv("rapids/lln_37GHz_2/output/ascii/attenuation_total.csv",skiprows=7,names=field_names)

rapids_total_attenuation_37_LLN = pd.concat((file1,file2))

utils.plot_RAPIDS_outputs(rapids_total_attenuation_37_LLN,"Total attenuation for LLN at 37 GHz")

### Rain attenuation at 37 GHz for lln

field_names = ["SITE","SATELLITE","FREQUENCY","ELEVATION","ELEVATION_PROBABILITY","PROBABILITY","HR","R001","K","ALPHA","ATTENUATION"] #for rain

file1 = pd.read_csv("rapids/lln_37GHz_1/output/ascii/attenuation_rain.csv",skiprows=7,names=field_names)
file2 = pd.read_csv("rapids/lln_37GHz_2/output/ascii/attenuation_rain.csv",skiprows=7,names=field_names)

rapids_rain_attenuation_37_LLN = pd.concat((file1,file2))

utils.plot_RAPIDS_outputs(rapids_rain_attenuation_37_LLN,"Rain attenuation for LLN at 37 GHz")

In [ ]:
import warnings
warnings.filterwarnings("ignore")

### compute the difference between LLN and LUB for total attenuation at 37 GHz

for i in range(0,18):
    
    # Ensure the ATTENUATION column is of string type
    rapids_total_attenuation_37_LLN['ATTENUATION'].iloc[i*26:(i+1)*26] = rapids_total_attenuation_37_LLN['ATTENUATION'].iloc[i*26:(i+1)*26].astype(str)
    rapids_total_attenuation_37_LUB['ATTENUATION'].iloc[i*26:(i+1)*26] = rapids_total_attenuation_37_LUB['ATTENUATION'].iloc[i*26:(i+1)*26].astype(str)

    # Remove ' dB' and convert to float
    rapids_total_attenuation_37_LLN['ATTENUATION'].iloc[i*26:(i+1)*26] = rapids_total_attenuation_37_LLN['ATTENUATION'].iloc[i*26:(i+1)*26].str.replace(' dB','').astype(float)
    rapids_total_attenuation_37_LUB['ATTENUATION'].iloc[i*26:(i+1)*26] = rapids_total_attenuation_37_LUB['ATTENUATION'].iloc[i*26:(i+1)*26].str.replace(' dB','').astype(float)

    LLN_LUB_37_tot_difference = rapids_total_attenuation_37_LLN['ATTENUATION'].iloc[i*26:(i+1)*26] - rapids_total_attenuation_37_LUB['ATTENUATION'].iloc[i*26:(i+1)*26]
    
    plt.semilogx(rapids_total_attenuation_37_LUB["PROBABILITY"].values[i*26:(i+1)*26],LLN_LUB_37_tot_difference,label="{}°".format((i+1)*5))

plt.title("Difference between LLN and LUB for total attenuation at 37 GHz")
plt.xlim(1e-3,100)
plt.xlabel("Probability of occurence")
plt.ylabel("Attenuation difference (dB)")
plt.legend()
plt.show()

### compute the difference between LLN and LUB for rain attenuation at 37 GHz

for i in range(0,18):
    
    # Ensure the ATTENUATION column is of string type
    rapids_rain_attenuation_37_LLN['ATTENUATION'].iloc[i*26:(i+1)*26] = rapids_rain_attenuation_37_LLN['ATTENUATION'].iloc[i*26:(i+1)*26].astype(str)
    rapids_rain_attenuation_37_LUB['ATTENUATION'].iloc[i*26:(i+1)*26] = rapids_rain_attenuation_37_LUB['ATTENUATION'].iloc[i*26:(i+1)*26].astype(str)
    
    # Remove ' dB' and convert to float
    rapids_rain_attenuation_37_LLN['ATTENUATION'].iloc[i*26:(i+1)*26] = rapids_rain_attenuation_37_LLN['ATTENUATION'].iloc[i*26:(i+1)*26].str.replace(' dB','').astype(float)
    rapids_rain_attenuation_37_LUB['ATTENUATION'].iloc[i*26:(i+1)*26] = rapids_rain_attenuation_37_LUB['ATTENUATION'].iloc[i*26:(i+1)*26].str.replace(' dB','').astype(float)
    
    LLN_LUB_37_rain_difference = rapids_rain_attenuation_37_LLN['ATTENUATION'].iloc[i*26:(i+1)*26] - rapids_rain_attenuation_37_LUB['ATTENUATION'].iloc[i*26:(i+1)*26]
    
    plt.semilogx(rapids_rain_attenuation_37_LUB["PROBABILITY"].values[i*26:(i+1)*26],LLN_LUB_37_rain_difference,label="{}°".format((i+1)*5))
    
plt.title("Difference between LLN and LUB for rain attenuation at 37 GHz")
plt.xlim(1e-3,100)
plt.xlabel("Probability of occurence")
plt.ylabel("Attenuation difference (dB)")
plt.legend()
plt.show()